In [1]:
import os
from data import read_examples

data_path = './RACE/'
train_high, train_middle = read_examples(os.path.join(data_path, 'train'))

In [2]:
i = 3

print("id: \n", train_high[i].id)
print("passage: \n",train_high[i].passage)
print("questions: \n",train_high[i].question)
print("options: \n",train_high[i].options)
print("answers: \n",train_high[i].answers)

id: 
 high1000.txt
passage: 
 When newspapers and radio describe the damage caused by a hurricane  named Hazel, girls named Hazel are probably teased  by their friends. To keep out of trouble, the Weather Bureau says,"Any _ between hurricane names and the names of particular girls is purely accidental."
Some women became angry because hurricanes are given their names, but many other women are proud to see their names make headlines. They don't even care that they are the names of destructive storms. Because more women seem to like it than dislike it, the Weather Bureau has decided to continue using girl's names for hurricanes.
In some ways a hurricane is like a person. After it is born, it grows and develops, then becomes old and dies. Each hurricane has a character of its own. Each follows its own path through the world, and people remember it long after it gone. So it is natural to give hurricanes' names, and to talk about them almost if they were alive.
questions: 
 ['What happens t

In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

arguments = dict(seq_length=512)

#preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
preprocess = hub.load('https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3')
bert_preprocess = hub.KerasLayer(preprocess)

#bert = hub.load('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-2_H-128_A-2/1')
bert = hub.load('https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-256_A-4/2')
bert_layer = hub.KerasLayer(bert)

In [4]:
from preprocessing import dcmn_preprocess

sample_size = 1000
sample_data = train_high[:1000]
validation_data = train_high[sample_size:sample_size + 100]

sample_inputs, sample_answers = dcmn_preprocess(sample_data)
valid_inputs, valid_answers = dcmn_preprocess(sample_data)
answer_dict = {'A': tf.constant([1, 0, 0, 0]),
               'B': tf.constant([0, 1, 0, 0]),
               'C': tf.constant([0, 0, 1, 0]),
               'D': tf.constant([0, 0, 0, 1])}
sample_answers_encoded = tf.convert_to_tensor([answer_dict[a] for a in sample_answers])
valid_answers_encoded = tf.convert_to_tensor([answer_dict[a] for a in valid_answers])

In [7]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.layers import Dense
from model import DCMN, GateLayers

config = {'n_choices': 4,
          'hidden_size': 256, #Under current setup, must adjust to match bert embedding size
          'dropout': 0.1,
          'bert_preprocess': bert_preprocess,
          'bert': bert_layer,
          'gate_layer': GateLayers,
          'classifier': Dense(4, activation='softmax')}

model = DCMN(config)

In [8]:
model.compile(
    optimizer = tf.keras.optimizers.Adam(5e-3),
    loss = tf.keras.losses.CategoricalCrossentropy(),
    metrics = [tf.keras.metrics.CategoricalAccuracy()]
)

model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 5, validation_data=(valid_inputs, valid_answers_encoded))

Epoch 1/5
325/325 [==============================] - 69s 193ms/step - loss: 1.7241 - categorical_accuracy: 0.2510 - val_loss: 1.4667 - val_categorical_accuracy: 0.3059
Epoch 2/5
325/325 [==============================] - 63s 194ms/step - loss: 1.6140 - categorical_accuracy: 0.2818 - val_loss: 1.5714 - val_categorical_accuracy: 0.2895
Epoch 3/5
325/325 [==============================] - 62s 191ms/step - loss: 1.6178 - categorical_accuracy: 0.2932 - val_loss: 1.5602 - val_categorical_accuracy: 0.2982
Epoch 4/5
325/325 [==============================] - 63s 193ms/step - loss: 1.6365 - categorical_accuracy: 0.3068 - val_loss: 1.4048 - val_categorical_accuracy: 0.3478
Epoch 5/5
325/325 [==============================] - 64s 198ms/step - loss: 1.5418 - categorical_accuracy: 0.3090 - val_loss: 1.3041 - val_categorical_accuracy: 0.3932


In [ ]:
model.fit(x = sample_inputs, y = sample_answers_encoded, batch_size = 10, epochs = 10, validation_data=(valid_inputs, valid_answers_encoded))